In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
qa = pipeline("question-answering")
sentiment = pipeline("sentiment-analysis")

In [ ]:
beige_df = pd.read_csv('/content/drive/MyDrive/beigebook2002-2022(2).csv') # 베이지북 파일 불러오기

date_list = beige_df['date'].tolist()
beige_list = beige_df['beige_book'].tolist()  # 베이지북 데이터 전처리

# qa 및 감성분석

data = []
for i in range(len(beige_list)):
  a = qa(question = 'What is the turnover rate in the US?', context = beige_list[i])
  # Can US economy grow or not?
  senti = sentiment(a.get('answer'))
  if senti[0]['label'] == 'NEGATIVE':
    senti[0]['score'] = senti[0]['score'] * (-1)
  data.append([a['answer'],a['score'],senti[0]['score'],senti[0]['label']])
  result = pd.DataFrame(data, columns=['answer', 'ans_score', 'senti_score', 'label'])
    
## 분석 결과만 데이터프레임화화

quarterly = beige_df['year'].astype(str)+'Q'+beige_df['quarter'].astype(str)
from statsmodels.tsa.base.datetools import dates_from_str

quarterly = dates_from_str(quarterly)
quarterly

result['quarterly'] = quarterly


df_fed = result.query("quarterly >= '2010Q3'")
fed = df_fed.groupby('quarterly').mean()
fed

# 분석결과 전처리
score_df2 = pd.DataFrame()
score_df2['Beige1'] = data6['senti_score']

score_df2['date'] = data6['quarterly']
score_df2['date'] = pd.to_datetime(score_df2['date'],format='%Y-%m-%d')

score_df2['year'] = score_df2['date'].dt.year
score_df2['quarter'] = score_df2['date'].dt.quarter
score_df2 = score_df2.groupby(by=[score_df2['year'],score_df2['quarter']]).mean()
score_df2

In [ ]:
# 각 컬럼을 섹션으로 나눠 평균 계산 
def averaging(column):
    section1 = []
    section2 = []
    section3 = []
    for i in range(len(df)):
        if i <= 24:
            section1.append(df[column][i])
        elif i <= 37:
            section2.append(df[column][i])
        else:
            section3.append(df[column][i])
    df[column][:25] = np.mean(section1)
    df[column][25:38] = np.mean(section2)
    df[column][38:] = np.mean(section3)
    return df

averaging('testimony')
averaging('minutes')
averaging('beige')
averaging('speech1')
averaging('speech2')